# Examples from (arxiv) Appendix B

In this notebook we are going to show the automated computations to produce the results on Appendix B on the arXiv version of [this paper](https://arxiv.org/abs/2202.05550). These examples explored other sequences obtained from OEIS and have several identities we could prove. 

These examples were taken through exploration from [OEIS](https://oeis.org/). We encourage the reader to see the script `oeis_information_getter.py` in this folder to see the exploration technique and the sequences (which include those shown in this notebook) obtained from this database.

In [1]:
%display latex
from alive_progress import alive_bar
from pseries_basis import *
from pseries_basis.ore import *
from pseries_basis.io import *
from ore_algebra import *
# Creating the objects used in the notebook
OE, (x, E) = get_recurrence_algebra("x", "E")
B = BinomialBasis()

## The example [A007317](https://oeis.org/A007317) (Example 56)

This sequence (the direct binomial transform of the Catalan numbers) starts with the terms $0, 1, 2, 5, 15, 51, 188,\ldots$ and has the following recurrence:
$$(n+2)a_{n+2} = (6n+4)a_{n+1} - 5na_n.$$

In [2]:
A007317 = EnhOEISSequence("A007317")
L007317 = A007317.dfinite_recurrence()
## Removing the offset of the sequence
L007317 = L007317.annihilator_of_associate(E**A007317.offsets()[0])
A007317_seq = solution(L007317, A007317.first_terms(required_init(L007317)))

First, we can check that this, indeed the binomial transform of the Catalan numbers. For doing so, we compute a sequence $b_k$ such that:
$$a_n = \sum_{k=0}^n b_k \binom{n}{k},$$
obtaining the following recurrence for $b_n$:

In [3]:
equ_b = B.recurrence_orig(L007317); show(equ_b)
seq_b = solution(equ_b,B.evaluation_to_self(A007317_seq, required_init(equ_b)))
seq_b[:10]

(-x - 3)*E^2 + (3*x + 4)*E + 4*x + 2

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

On the other hand, the Catalan numbers satisfies the following recurrence:


In [4]:
equ_cat = (x+2)*E - 2*(2*(x+1)-1); equ_cat

(x + 2)*E - 4*x - 2

Using closure properties, we can check that $b_n$ is, indeed, the sequence of Catalan numbers:

In [5]:
d = required_init(equ_b.lclm(equ_cat))
print(f"We need to check equality up to {d} elements")
all(seq_b(i) == catalan_number(i) for i in range(d+1))

We need to check equality up to 4 elements


True

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [6]:
equ_c = B.recurrence_orig(equ_b); show(equ_c)
seq_c = solution(equ_c,B.evaluation_to_self(seq_b, required_init(equ_c)))
[seq_c(i) for i in range(10)]

(-x - 4)*E^3 - 2*E^2 + (7*x + 10)*E + 6*x + 6

[1, 0, 1, 1, 3, 6, 15, 36, 91, 232]

We can try to see if this new sequence appear in OEIS. In fact, id does and it provides a formula:

In [7]:
oeis([seq_c(i) for i in range(20)])

0: A005043: Riordan numbers: a(n) = (n-1)*(2*a(n-1) + 3*a(n-2))/(n+1).
1: A099323: Expansion of (sqrt(1+3*x) + sqrt(1-x))/(2*sqrt(1-x)).
2: A174297: First column of A174295.

We have three possible candidates for our sequence $(c_n)$. Let us check the first (Riordan numbers) using closure properties again:

In [8]:
riordan = oeis([seq_c(i) for i in range(20)])[0]
equ_riordan = (x+3)*E^2 - (x+1)*2*E - (x+1)*3
d = required_init(equ_c.lclm(equ_riordan))
print(f"We need to check equality up to {d} elements")
[seq_c(i) for i in range(d+1)] == list(riordan.first_terms(d+1))

We need to check equality up to 5 elements


True

Hence, we conclude that the sequence $a_n$ that satisfies the recurrence
$$(n+3)a_{n+2} = (6n+10)a_{n+1} - 5(n+1)a_n,$$
can be written by:
$$a_n = \sum_{k=0}^n c_k \binom{n}{k} = \sum_{k=0}^n \sum_{l=0}^k R_l \binom{k}{l} \binom{n}{k}.$$

## The example [A000172](https://oeis.org/A000172) (Example 57)

This sequence (also known as Franel numbers) starts with the terms $1, 2, 10, 56, 346, 2252,\ldots$ and satisfies the following recurrence:
$$(n + 1)^2a_{n+1} = (7n^2 + 7n + 2)a_n + 8n^2a_{n-1},\text{ for $n > 1$}$$


In [9]:
A000172 = EnhOEISSequence("A000172")
L000172 = A000172.dfinite_recurrence()
## Removing the offset of the sequence
L000172 = L000172.annihilator_of_associate(E**A000172.offsets()[0])
A000172_seq = solution(L000172, A000172.first_terms(required_init(L000172)));
L000172

(x^2 + 4*x + 4)*E^2 + (-7*x^2 - 21*x - 16)*E - 8*x^2 - 16*x - 8

First of all, we can check that this sequence $a_n$ satisfies the following formula:
$$a_n = \sum_{k=0}^n \binom{n}{k}^3,$$
by using `ProductBasis` within the package, we can obtain a recurrence for a sequence $b_k$ such that:
$$a_n = \sum_{k=0}^n b_k \binom{n}{k}^3$$
using the following piece of code:

In [15]:
B3 = ProductBasis([B,B,B], ends=["E"]); B3
M = B3.recurrence(L000172)
fM = [B3.remove_Sni(M.coefficient((i,0))) for i in range(3)]
new_eq = fM[0].gcrd(*fM[1:])
new_eq

Sn - 1

Hence, the desired formula with the cube of the binomial holds.

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [23]:
equ_d = B.recurrence_orig(L000172)
seq_d = solution(equ_d,B.evaluation_to_self(A000172_seq, required_init(equ_d))) 
equ_c = B.recurrence_orig(equ_d); show(equ_c)
seq_c = solution(equ_c,B.evaluation_to_self(seq_d, required_init(equ_c)))
seq_c[:10]

(x^2 + 12*x + 36)*E^6 + (3*x^2 + 29*x + 70)*E^5 + (-23*x^2 - 236*x - 600)*E^4 + (-135*x^2 - 1115*x - 2298)*E^3 + (-266*x^2 - 1738*x - 2820)*E^2 + (-228*x^2 - 1092*x - 1272)*E - 72*x^2 - 216*x - 144

[1, 0, 6, 12, 90, 360, 2040, 10080, 54810, 290640]

We obtain an order 6 recurrence for this sequence. Can we simplify this recurrence?

In [21]:
guess([seq_c(i) for i in range(50)], OE)

(-x^2 - 6*x - 9)*E^3 + (x^2 + 5*x + 6)*E^2 + (24*x^2 + 96*x + 96)*E + 36*x^2 + 108*x + 72

And is this sequence back in OEIS?

In [27]:
oeis(seq_c[:10])

0: A002898: Number of n-step closed paths on hexagonal lattice.

## The example [A258431](https://oeis.org/A258431) (Example 58)

This sequence starts with the terms $0, 1, 5, 23, 102, 443, 1898, 8054,\ldots$ and counts the **_sum over all peaks of Dyck paths of semilength n of the arithmetic mean of the x and y coordinates_**. It satisfies the following recurrence equation:
$$(n-1)a_n = (8n-10)a_{n-1}-(16n-24)a_{n-2}\text{ for $n > 2$}.$$

In [30]:
A258431 = EnhOEISSequence("A258431")
L258431 = E*((x+1)*E^2 - (8*(x+2)-10)*E + (16*(x+2)-24))
A258431_seq = solution(L258431, [0,1,5])

In [31]:
A258431_seq[:20]

[0,
 1,
 5,
 23,
 102,
 443,
 1898,
 8054,
 33932,
 142163,
 592962,
 2464226,
 10209620,
 42190558,
 173962532,
 715908428,
 2941192472,
 12065310083,
 49428043442,
 202249741418]

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [45]:
equ_d = B.recurrence_orig(L258431)
seq_d = solution(equ_d,B.evaluation_to_self(A258431_seq, required_init(equ_d))) 
equ_c = B.recurrence_orig(equ_d); show(equ_c)
seq_c = solution(equ_c,B.evaluation_to_self(seq_d, required_init(equ_c)))
seq_c[:10]

(x + 4)*E^5 + (x + 1)*E^4 + (-8*x - 26)*E^3 + (-8*x - 8)*E^2 + (16*x + 40)*E + 16*x + 16

[0, 1, 1, 5, 6, 23, 30, 102, 140, 443]

We obtain an order 5 recurrence for this sequence. Can we simplify this recurrence?

In [46]:
guess([seq_c(i) for i in range(50)], OE)

(-x - 3)*E^4 + 2*E^3 + (8*x + 16)*E^2 - 8*E - 16*x - 16

And is this sequence back in OEIS?

In [47]:
len(oeis([seq_c(i) for i in range(1,10)]))

0

This is not very nice. However, something strange happens: the original sequence appears as a subsequence of $c_k$:

In [38]:
[seq_c(2*i-1) for i in range(10)] == [A258431(i) for i in range(10)]

True

Which shows that, for the solution to 
$$(n-1)a_n = (8n-10)a_{n-1}-(16n-24)a_{n-2}\text{ for $n > 2$}, a(0) = 0, a(1) = 1, a(2) = 5;$$
if we write this sequence with
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k},$$
then we conclude that:
$$c_{2n-1} = a_n\text{ for all $n \in \mathbb{N}$}.$$
This final identity can be proven by closure properties of D-finite sequences.

## The example [A032443](https://oeis.org/A032443) (Example 59)

This sequence starts with the terms $1, 3, 11, 42, 163, 638, 2510, 9908,\ldots$ and has the following formula:
$$a_n = \sum_{i=0}^n \binom{2n}{i},$$
satisfying the following recurrence:
$$na_n + 2(-4n+3)a_{n-1} + 8(2n-3)a_{n-2} = 0$$

In [44]:
A032443 = EnhOEISSequence("A032443")
L032443 = A032443.dfinite_recurrence(); show(L032443)
A032443_seq = A032443.sequence

(x + 2)*E^2 + (-8*x - 10)*E + 16*x + 8

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence_orig` in the package `pseries_basis` we can get the new equation for $c_k$:

In [48]:
equ_d = B.recurrence_orig(L032443)
seq_d = solution(equ_d,B.evaluation_to_self(A032443_seq, required_init(equ_d))) 
equ_c = B.recurrence_orig(equ_d); show(equ_c)
seq_c = solution(equ_c,B.evaluation_to_self(seq_d, required_init(equ_c)))
seq_c[:10]

(x + 4)*E^4 + (-x - 3)*E^3 + (-6*x - 16)*E^2 + (4*x + 8)*E + 8*x + 8

[1, 1, 3, 4, 11, 16, 42, 64, 163, 256]

We can try to see if this new sequence appear in OEIS. In fact, id does and it provides a formula:

In [49]:
oeis([seq_c(i) for i in range(20)])

0: A027306: a(n) = 2^(n-1) + ((1 + (-1)^n)/4)*binomial(n, n/2).

Hence we can conclude that the following identity holds:
$$\sum_{k=0}^n \binom{2n}{k} = \sum_{k=0}^n \sum_{l=0}^k\left[ \left(2^{l-1} + \frac{1 + (-1)^l}{4}\binom{l}{l/2}\right) \binom{k}{l}\binom{n}{k}\right]$$

In [50]:
## Checking identity for i = 0,...,30
lhs = lambda n : sum(binomial(2*n, k) for k in range(n+1))
rhs = lambda n : sum(sum((2**(l-1) + (1/2*binomial(l,l/2) if l%2 == 0 else 0)) * binomial(k,l) * binomial(n,k) for l in range(k+1)) for k in range(n+1))
all(lhs(i) == rhs(i) for i in range(30))

True

## Example from a Convolution

We know that the convolution of $n!$ and $1/n!$ is not Liouvillian and satisfies the following recurrence:
$$(n+3)y_{n+3} - (n^2 + 6n + 10)y_{n+2} + (2n + 5)y_{n+1} - y_n  = 0.$$

Let us see what happen if we apply the binomial basis to this recurrence:

In [73]:
L_con_facN_ifacN = (x+3)*E^3 - (x^2 + 6*x +10)*E^2 + (2*x+5)*E - 1
con_facN_ifacN = solution(L_con_facN_ifacN, [sum(QQ(factorial(k)/factorial(n-k)) for k in range(n+1)) for n in range(4)])
con_facN_ifacN[:10]

[1,
 2,
 7/2,
 26/3,
 749/24,
 8843/60,
 614227/720,
 3669289/630,
 1844279257/40320,
 10519507811/25920]

We check this sequence is, exactly, the desired convolution:

In [101]:
real_con_facN_ifacN = LambdaSequence(lambda n : sum(QQ(factorial(k)/factorial(n-k)) for k in range(n+1)), QQ)
(real_con_facN_ifacN + (-con_facN_ifacN)).almost_zero(15)

True

##### Double sum part
Let us describe the values of $y_n$ as a double sum of binomials of the form:
$$y_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence_orig` in the package `pseries_basis` we can get the new equation for $c_k$:

In [102]:
equ_d = B.recurrence_orig(L_con_facN_ifacN)
seq_d = solution(equ_d,B.evaluation_to_self(con_facN_ifacN, required_init(equ_d))) 
equ_c = B.recurrence_orig(equ_d); show(equ_c)
seq_c = solution(equ_c,B.evaluation_to_self(seq_d, required_init(equ_c)))
seq_c[:10]

(x + 7)*E^7 + (-x^2 - 4*x + 12)*E^6 + (-10*x^2 - 79*x - 144)*E^5 + (-41*x^2 - 326*x - 653)*E^4 + (-88*x^2 - 624*x - 1125)*E^3 + (-104*x^2 - 612*x - 911)*E^2 + (-64*x^2 - 288*x - 320)*E - 16*x^2 - 48*x - 32

[1,
 0,
 -1/2,
 11/3,
 -17/8,
 899/30,
 13007/144,
 665419/840,
 34410911/5760,
 2439240803/45360]

We obtain an order 7 recurrence for this sequence. Can we simplify this recurrence?

In [103]:
guess([seq_c(i) for i in range(50)], OE)

(x + 5)*E^5 + (-x^2 - 2*x + 8)*E^4 + (-8*x^2 - 40*x - 47)*E^3 + (-24*x^2 - 116*x - 143)*E^2 + (-32*x^2 - 128*x - 128)*E - 16*x^2 - 48*x - 32

## The example [A001472](https://oeis.org/A001472)

This sequence starts with the terms $1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616, 218656,\ldots$ and counts the number of degree-$n$ permutations of order dividing 4. It satisfies a recurrence: 
$$a(0)=1, a(1)=1, a(2)=2, a(3)=4,$$
$$a(n) = a(n-1) + (n-1)a(n-2) + (n^3-6n^2+11n-6)a(n-4)$$

In [113]:
A001472 = EnhOEISSequence("A001472")
L001472 = E^4 - E^3 - (x+3)*E^2 - ((x+4)^3 - 6*(x+4)^2 + 11*(x+4) - 6) ; show(L001472)
A001472_seq = solution(L001472, A001472.first_terms(required_init(L001472)))
(A001472_seq + (-A001472.sequence)).almost_zero(20)

E^4 - E^3 + (-x - 3)*E^2 - x^3 - 6*x^2 - 11*x - 6

True

In [114]:
A001472_seq[:20]

[1,
 1,
 2,
 4,
 16,
 56,
 256,
 1072,
 6224,
 33616,
 218656,
 1326656,
 9893632,
 70186624,
 574017536,
 4454046976,
 40073925376,
 347165733632,
 3370414011904,
 31426411211776]

This sequence has a formula given as follows:

$$a(n) = n!\sum_{k=1}^n \frac{1}{k!}\left(\sum_{j=\lfloor(4*k-n)/3\rfloor}^k\binom{k}{j} \binom{j}{n-4k+3j}(1/2)^{n-4k+3j}(1/4)^{k-j}\right)$$

We now build the sequence that has $a_n$ as the inner summand of a doulbe binomial sum:

In [115]:
new_sequence = LambdaSequence(lambda n : sum(sum(A001472_seq[l]*B[l](k)*B[k](n) for l in range(k+1)) for k in range(n+1)), ZZ)
guessed_recurrence = guess(new_sequence[:100], OE)

In [117]:
show(guessed_recurrence)
new_sequence[:10]

-E^4 + 3*E^3 + (x + 3)*E^2 + x^3 + 6*x^2 + 11*x + 6

[1, 3, 10, 36, 144, 648, 3264, 18000, 107088, 682992]

#### Writing the example

Consider the sequence $(1,3,10,36,144,648,3264,18000,107088,682992,\ldots)$ defined as solution to the recurrence equation:
$$y_{n+4} - 3 y_{n+3} + \left(-n - 3\right) y_{n+2} - (n^{3} + 6 n^{2} + 11 n + 6)y_n = 0$$

This sequence does not show up in OEIS, but if we write it in the following form:
$$y_n = \sum_{k=0}^n c_k \binom{n}{k},$$
then the sequence $(c_n)_n$ also satisfies a recurrence equation given by:

In [120]:
equ_y = -guessed_recurrence
seq_y = solution(equ_y, [1,3,10,36, 144])
equ_c = B.recurrence_orig(equ_y); show(equ_c)
seq_c = solution(equ_c,B.evaluation_to_self(seq_y, required_init(equ_c))) 
seq_c[:10]

E^7 + E^6 + (-x - 9)*E^5 + (-3*x - 20)*E^4 + (-x^3 - 15*x^2 - 77*x - 134)*E^3 + (-3*x^3 - 36*x^2 - 142*x - 183)*E^2 + (-3*x^3 - 27*x^2 - 78*x - 72)*E - x^3 - 6*x^2 - 11*x - 6

[1, 2, 5, 14, 49, 202, 949, 4790, 26513, 159218]

This sequence does not appear in OEIS still. The order has gotten bigger and we still have no information. We can try to apply the transformation once again:
$$c_n = \sum_{k=0}^nd_k\binom{n}{k}.$$ 
This sequence $(d_n)_n$ is also D-finite. Let see the recurrence and solution:

In [121]:
equ_d = B.recurrence_orig(equ_c); show(equ_c)
seq_d = solution(equ_d,B.evaluation_to_self(seq_c, required_init(equ_d))) 
seq_d[:10]

E^7 + E^6 + (-x - 9)*E^5 + (-3*x - 20)*E^4 + (-x^3 - 15*x^2 - 77*x - 134)*E^3 + (-3*x^3 - 36*x^2 - 142*x - 183)*E^2 + (-3*x^3 - 27*x^2 - 78*x - 72)*E - x^3 - 6*x^2 - 11*x - 6

[1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616]

This sequence does appear in OEIS as the sequence [A001472](https://oeis.org/A001472). Sequence A001472 is D-finite with recurrence
$$a(n) = a(n-1) + (n-1)a(n-2) + (n^3-6n^2+11n-6)a(n-4).$$
Using closure properties, we can check if these two sequences are the same:

In [126]:
diff_equ = L001472.lclm(equ_d)
diff_seq = solution(diff_equ, (A001472.sequence + (-seq_d))[:required_init(diff_equ)])
show(diff_equ)
[diff_seq(i) for i in range(required_init(diff_equ)+5)]

E^10 + 8*E^9 + (-x + 15)*E^8 + (-9*x - 42)*E^7 + (-x^3 - 24*x^2 - 224*x - 732)*E^6 + (-9*x^3 - 189*x^2 - 1377*x - 3432)*E^5 + (-33*x^3 - 594*x^2 - 3597*x - 7288)*E^4 + (-63*x^3 - 945*x^2 - 4698*x - 7712)*E^3 + (-66*x^3 - 792*x^2 - 3110*x - 3984)*E^2 + (-36*x^3 - 324*x^2 - 936*x - 864)*E - 8*x^3 - 48*x^2 - 88*x - 48

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Sequnce A001472 has a closed form formula. Hence, we can write the solutions to our original equation
$$y_{n+4} - 3 y_{n+3} + \left(-n - 3\right) y_{n+2} - (n^{3} + 6 n^{2} + 11 n + 6)y_n = 0$$
with the following formula:
$$y_n = \sum_{k=0}^n 
    \sum_{l=0}^k 
        \left(l!\sum_{p=1}^l 
            \frac{1}{p!}\left(
                \sum_{j=\lfloor(4p-l)/3\rfloor}^p\binom{p}{j} \binom{j}{l-4p+3j}(1/2)^{l-4p+3j}(1/4)^{p-j}
            \right)
        \right)
\binom{k}{l}\binom{n}{k}$$